<b>Note: This Jupyter Notebook is associated with the article [The Stock Market Under Trump: A Hypothesis Based on Former Republican Presidencies](https://alpaca.markets/learn/the-stock-market-under-trump-a-hypothesis-based-on-former-republican-presidencies).</b>

# Step 1: Setting Up the Environment and Trade Parameters

In [ ]:
# Install or upgrade the package `polygon-api-client` and `plotly` and import them
!python3 -m pip install --upgrade polygon-api-client
!python3 -m pip install --upgrade plotly

#import modules
from polygon import RESTClient
import datetime as dt
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
from datetime import datetime, timedelta

# Create client and authenticate with Polygon.io
# [Reference](https://github.com/polygon-io/client-python)
# Add your key to Colab Secrets. Add your API key to the Colab Secrets manager to securely store it
from google.colab import userdata
polygonAPIkey = userdata.get('POLYGON_API_KEY')
client = RESTClient(polygonAPIkey) # api_key is used

def calculate_days_between(from_date, to_date):
    from_date_obj = datetime.strptime(from_date, '%Y-%m-%d')
    to_date_obj = datetime.strptime(to_date, '%Y-%m-%d')
    return (to_date_obj - from_date_obj).days

# Step 2: Historical Market Data Analysis with Polygon.io API

In [ ]:
# Daily bars of target stocks (one of LMT, RTX, NOC). The code below checks "LMT" stock.
dataRequest = client.get_aggs(ticker="LMT",
                              multiplier=1,
                              timespan='day',
                              from_='2017-01-01',
                              to='2021-01-31')

# List of polygon agg objects to DataFrame
priceData = pd.DataFrame(dataRequest)

# Create Date column
priceData['date'] = pd.to_datetime(priceData['timestamp'], unit='ms')

priceData = priceData.set_index('date')

# Generate plotly figure
fig = go.Figure(data=[go.Candlestick(x=priceData.index,
                open=priceData['open'],
                high=priceData['high'],
                low=priceData['low'],
                close=priceData['close'])])

# Open figure directly in the Colab notebook
fig.show()

In [ ]:
# Fetch the stock data from the Polygon API
def get_stock_data(underlying_symbol, from_date, to_date):
    # Get daily bars for the underlying stock
    req = client.get_aggs(ticker=underlying_symbol,
                          multiplier=1,             # 1-day timespan
                          timespan='day',
                          from_=from_date,
                          to=to_date)

    # List of polygon agg objects to DataFrame
    priceData = pd.DataFrame(req)
    # Create Date column
    priceData['date'] = pd.to_datetime(priceData['timestamp'], unit='ms')
    priceData = priceData.set_index('date')

    return priceData

# Calculate True Range for the underlying stock in the last 90 days
def calculate_true_range(data):
    data['prev_close'] = data['close'].shift(1)  # Shift the closing price by 1 day
    data['tr'] = np.maximum(
        data['high'] - data['low'],
        np.maximum(
            abs(data['high'] - data['prev_close']),
            abs(data['low'] - data['prev_close'])
        )
    )
    return data

# Calculate ATR for the underlying stock on the last 90 days
def calculate_atr(data, period=14):
    data['atr'] = data['tr'].rolling(window=period).mean()
    return data

# Calculate ATR for multiple stocks
def get_atr_for_stocks(stock_symbols, period, from_date, to_date):
    results = {}

    for symbol in stock_symbols:
        print(f"Processing {symbol}...")
        data = get_stock_data(symbol, from_date, to_date)
        data = calculate_true_range(data)
        data = calculate_atr(data, period=period)
        results[symbol] = data[['tr', 'atr']].dropna()  # Store TR and ATR data for each stock

    return results

# List of stock symbols
stock_symbols = ['LMT', 'RTX', 'NOC']

# Call the function for multiple stocks
atr_results = get_atr_for_stocks(stock_symbols, period=14, from_date='2024-01-05', to_date='2024-12-31')

# Display ATR results for each stock
for symbol, result in atr_results.items():
    # Calculate the mean ATR for the past days
    atr_mean = result["atr"].mean()
    days = calculate_days_between('2024-01-01', '2024-12-31')
    print(f"\nThe average ATR for {symbol} over the past {days} days is:\n")
    print(atr_mean)

# Step 3: Sell Cash-Secured Puts (CSPs)

In [ ]:
# Install or upgrade the package `alpaca-py` and import it
!python3 -m pip install --upgrade alpaca-py

import alpaca
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetOptionContractsRequest, MarketOrderRequest
from alpaca.data.requests import OptionLatestQuoteRequest, OptionSnapshotRequest
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestTradeRequest

# API credentials for Alpaca
# API_KEY = "YOUR_ALPACA_API_KEY_FOR_PAPER_TRADING"
# API_SECRET = 'YOUR_ALPACA_API_SECRET_KEY_FOR_PAPER_TRADING'
# Add your key to Colab Secrets. Add your API key to the Colab Secrets manager to securely store it
from google.colab import userdata
API_KEY = userdata.get('ALPACA_API_KEY')
API_SECRET = userdata.get('ALPACA_SECRET_KEY')
BASE_URL = None
## We use paper environment for this example
PAPER = True # Please do not modify this. This example is for paper trading only.

trade_client = TradingClient(api_key=API_KEY, secret_key=API_SECRET, paper=PAPER, url_override=BASE_URL)
stock_data_client = StockHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET)
option_historical_data_client = OptionHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET, url_override=BASE_URL)

# Set expiration range for options
today = datetime.now().date()

# Set the open interest volume threshold
OI_THRESHOLD = 100

# Buying power percentage to use for the trade
BUY_POWER_LIMIT = 0.10

# Check account buying power
buying_power = float(trade_client.get_account().buying_power)

# Calculate the limit amount of buying power to use for the trade
buying_power_limit = buying_power * BUY_POWER_LIMIT

# Define a 10% range around the underlying price
STRIKE_RANGE = 0.1

In [ ]:
# Set the expiration date range for the options
min_expiration = today + timedelta(days=7)
max_expiration = today + timedelta(days=35)

# Get the latest price of the underlying stock
def get_underlying_price(symbol):
    # Get the latest trade for the underlying stock
    underlying_trade_request = StockLatestTradeRequest(symbol_or_symbols=symbol)
    underlying_trade_response = stock_data_client.get_stock_latest_trade(underlying_trade_request)
    return underlying_trade_response[symbol].price

# Get the latest price of the underlying stock
for symbol in stock_symbols:
    # Get the latest price of the underlying stock
    underlying_price = get_underlying_price(symbol)
    print(f"\n{symbol} stock price: {underlying_price}")

In [ ]:
# Check for put options
def get_put_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration):

    # Fetch the options data to add to the portfolio
    req = GetOptionContractsRequest(underlying_symbols=[underlying_symbol],
                                    strike_price_gte=min_strike,
                                    strike_price_lte=max_strike,
                                    status="active",
                                    expiration_date_gte=min_expiration,
                                    expiration_date_lte=max_expiration,
                                    type="put",
                                    )

    # Get put option chain of the underlying symbol
    put_options = trade_client.get_option_contracts(req).option_contracts
    return put_options

In [ ]:
# Find the appropriate short put option to sell based on the delta, IV, and buying power limit
def find_short_put_option(put_options, buying_power_limit):

    short_put = None

    for option_data in put_options:
        try:
            # Ensure open_interest is valid and open_interest_date exists
            if option_data.open_interest is None or option_data.open_interest_date is None:
                # Skip options with missing open interest or quote data
                continue

            # Check open interest meets the threshold
            if float(option_data.open_interest) <= OI_THRESHOLD:
                print(f"Skipping option {option_data.symbol} due to lack of open interest. You may change the OI_THRESHOLD")
                continue

            # Get the latest quote for the option price
            option_symbol = option_data.symbol
            option_quote_request = OptionLatestQuoteRequest(symbol_or_symbols=option_symbol)
            option_quote = option_historical_data_client.get_option_latest_quote(option_quote_request)[option_symbol]

            # Extract option details
            option_price = (option_quote.bid_price + option_quote.ask_price) / 2
            option_size = float(option_data.size)
            strike_price = float(option_data.strike_price)

            # Fetch delta for each option
            # get option chain by underlying_symbol including IV, Greeks, and other information
            req = OptionSnapshotRequest(
                symbol_or_symbols = option_symbol
                )
            snapshot = option_historical_data_client.get_option_snapshot(req)
            delta = snapshot[option_symbol].greeks.delta
            iv = snapshot[option_symbol].implied_volatility
            print(f"delta is {delta} and IV is {iv}")

            # Check if delta is between -0.42 and -0.18, if IV is less than or equal to 60, and if the total contract exceeds the buying power limit
            if delta is not None and delta >= -0.42 and delta <= -0.18 and iv <= 60 and strike_price * option_size < buying_power_limit:

                # Create the appropriate short put option in the dictionary format
                short_put = {
                    'close_price': option_data.close_price,
                    'close_price_date': option_data.close_price_date,
                    'expiration_date': option_data.expiration_date,
                    'id': option_data.id,
                    'name': option_data.name,
                    'open_interest': option_data.open_interest,
                    'open_interest_date': option_data.open_interest_date,
                    'root_symbol': option_data.root_symbol,
                    'size': option_data.size,
                    'status': option_data.status,
                    'strike_price': option_data.strike_price,
                    'style': option_data.style,
                    'symbol': option_data.symbol,
                    'tradable': option_data.tradable,
                    'type': option_data.type,
                    'underlying_asset_id': option_data.underlying_asset_id,
                    'underlying_symbol': option_data.underlying_symbol,
                    'initial_delta': delta,
                    'initial_iv': iv,
                    'initial_option_price': option_price,
                }

                print(f"short_put is {short_put}")
            # If the short put is found, break the loop
            if short_put:
                break

        except KeyError as e:
            print(f"Missing data for option {option_symbol}: {e}")
            continue

    return short_put

In [ ]:
def execute_cash_secured_put(underlying_symbol, buying_power_limit, min_expiration, max_expiration):

    # Get the latest price of the underlying stock
    underlying_price = get_underlying_price(underlying_symbol)
    # Set the minimum and maximum strike prices based on the underlying price
    min_strike = str(underlying_price * (1 - STRIKE_RANGE))
    max_strike = str(underlying_price) # In cash-secured put, we explore options whose strike price is below the underlying price

    # Get put options
    put_options = get_put_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration)

    if put_options:
        # Get the latest price of the underlying stock
        underlying_price = get_underlying_price(symbol=underlying_symbol)

        # Find appropriate short put options
        short_put = find_short_put_option(put_options, buying_power_limit)

        # Proceed if short put options are found
        if short_put:

            # Place orders for the short put if credit received meets a threshold (e.g., minimum profit)
            req = MarketOrderRequest(
                symbol=short_put['symbol'],
                qty=1,
                side='sell',
                type='market',
                time_in_force='day'
            )

            trade_client.submit_order(req)

            success_message = (f"Placing Cash Secured Put on {underlying_symbol} successfully:\n"
                               f"Sell {short_put['initial_delta']} Delta Put: {short_put['symbol']} "
                               f"(Strike: {short_put['strike_price']}, Premium to Receive: {short_put['initial_option_price']})")
            return success_message, short_put

        else:
            return "Could not find suitable options which has delta between -20 and -40 and iv less than or equal to 60.", None
    else:
        return "No put options available for the underlying symbol.", None

In [ ]:
import concurrent.futures

stock_symbols = ['XOM', 'PAA', 'HAL']

# You can run the `find_short_put_option` function just to find the short put option.
def process_stock(symbol):
    # Get the latest price of the underlying stock
    underlying_price = get_underlying_price(symbol)
    min_strike = str(underlying_price * (1 - STRIKE_RANGE))
    max_strike = str(underlying_price)
    put_options = get_put_options(symbol, min_strike, max_strike, min_expiration, max_expiration)
    return find_short_put_option(put_options, buying_power_limit)

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_stock, stock_symbols))

print(results)